In [1]:
import vectorbt as vb
import numpy as np
import pandas as pd

import operator
from functools import reduce
from datetime import datetime
from utils import binance_resample, yf_resample
from signals import MomentumSignals, DrawdownSignals, DumpSignals, VolumeSignals, OscillatorSignals


start_date = datetime(2021, 6, 1)

vb.settings.array_wrapper['freq'] = 'd'
vb.settings.plotting['layout']['template'] = 'vbt_dark'
vb.settings.portfolio['init_cash'] = 1000.0
vb.settings.portfolio['fees'] = 0.001
vb.settings.portfolio['slippage'] = 0.002

In [2]:
# Get SPY data
spy_raw = vb.YFData.download('SPY', start=start_date, end='now UTC', interval='1h')
spy = yf_resample(spy_raw.data['SPY'], '4H')

In [3]:
use = 'BTCUSDT'

bdata = vb.BinanceData.download(use, start=start_date, end='now UTC', interval='1h')
data = pd.concat({i: binance_resample(x, '4H') for i, x in bdata.data.items()}, axis=1)

2021-05-31 17:00:00+00:00 - 2022-02-10 21:00:00+00:00: : 13it [00:08,  1.47it/s]


# Signal

In [5]:
price = data[f'{use}'].close
vol = data[f'{use}'].volume
logvol = np.log(1 + vol)

osig = OscillatorSignals(price)
msig = MomentumSignals(price, mom_period=24)
dsig = DumpSignals(price, ma_period=84, dma_period=3)
ddsig = DrawdownSignals(price)
vsig = VolumeSignals(logvol, ma_period=42, dma_period=3)

# Independent buy/sell signals
buy_signals = [
    msig.in_trough_signal(),
    msig.momentum_buy_signal(ma_period=12, thresh=0.95),
    dsig.recovery_buy_signal(),
    ddsig.drawdown_buy_signal(dd_period=84),
]

sell_signals = [
    msig.dying_momentum_signal(ma_period=84),
    msig.momentum_sell_signal(),
    dsig.dump_sell_signal(thres=1.1),
]

In [6]:
high_vol_only = False

buy_signal = reduce(operator.or_, buy_signals)
sell_signal = reduce(operator.or_, sell_signals)
if high_vol_only:
    buy_signal = vsig.vol_up_signal(1.05) & buy_signal
    sell_signal = vsig.vol_up_signal(1.05) & sell_signal

print(f'# Buy Signal : {buy_signal.sum()}')
print(f'# Sell Signal: {sell_signal.sum()}')

# Buy Signal : 90
# Sell Signal: 67


# Backtest

In [7]:
stop_loss = 0.1

portfolio = vb.Portfolio.from_signals(price, buy_signal, sell_signal, sl_stop=stop_loss, sl_trail=True)
portfolio.stats()

Start                         2021-05-31 16:00:00+00:00
End                           2022-02-10 20:00:00+00:00
Period                               1532 days 00:00:00
Start Value                                      1000.0
End Value                                    1646.33836
Total Return [%]                              64.633836
Benchmark Return [%]                           19.90122
Max Gross Exposure [%]                            100.0
Total Fees Paid                              141.642542
Max Drawdown [%]                              44.287218
Max Drawdown Duration                 648 days 00:00:00
Total Trades                                         44
Total Closed Trades                                  44
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  54.545455
Best Trade [%]                                24.627854
Worst Trade [%]                               -1

In [10]:
portfolio.trades.records.head(5)

,id,col,size,entry_idx,entry_price,entry_fees,exit_idx,exit_price,exit_fees,pnl,return,direction,status,parent_id
0,0,0,0.031446,47,31768.65048,0.999001,107,36355.10000,1.143227,142.083843,0.142226,0,1,0
1,1,0,0.032090,115,35554.68744,1.140943,123,32959.44900,1.057662,-85.479294,-0.074920,0,1,1
2,2,0,0.032393,124,32585.60112,1.055549,138,32814.21006,1.062954,5.286851,0.005009,0,1,2
3,3,0,0.032858,150,32285.18148,1.060831,173,36176.49202,1.188692,125.611651,0.118409,0,1,3
4,4,0,0.034700,182,34188.25002,1.186317,197,34539.26304,1.198497,9.795179,0.008257,0,1,4


In [1]:
portfolio.trades.plot()

NameError: name 'portfolio' is not defined